In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

np.__version__

/var/folders/b7/c67wpzpn25q0h072y9mmz_th0000gn/T/ipykernel_65523/1493306873.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


'1.26.3'

# Pull in data

In [3]:
PREVIOUS_OUTPUT_PATH = './data/22.2-v3-OUTPUT-data-through-sep-2023-ALL-SONGS-LABELLED-with-good-riddance-corrections.csv'
prev_df = pd.read_csv(PREVIOUS_OUTPUT_PATH)
prev_df['song_id'] = prev_df['song'] + prev_df['performer']
prev_df

,performer,song,love_song_sub_type,song_id
0,John Lennon,#9 Dream,NaN,#9 DreamJohn Lennon
1,Jay-Z Featuring Beyonce Knowles,'03 Bonnie & Clyde,It's Complicated,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles
2,Paul Davis,'65 Love Affair,NaN,'65 Love AffairPaul Davis
3,The Everly Brothers,('til) I Kissed You,Serenade,('til) I Kissed YouThe Everly Brothers
4,Nelson,(Can't Live Without Your) Love And Affection,Longing & Heartbreak,(Can't Live Without Your) Love And AffectionNe...
...,...,...,...,...
5136,SZA,Kill Bill,It's Complicated,Kill BillSZA
5137,Bizarrap & Shakira,"Bzrp Music Sessions, Vol. 53",Good Riddance,"Bzrp Music Sessions, Vol. 53Bizarrap & Shakira"
5138,PinkPantheress & Ice Spice,"Boy's A Liar, Pt. 2",It's Complicated,"Boy's A Liar, Pt. 2PinkPantheress & Ice Spice"
5139,Karol G x Shakira,TQG,Good Riddance,TQGKarol G x Shakira


In [4]:
# Fetch the correction gSheet
G_SHEET_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTwJo327jLAd-uMGiUeROIe7zrKt95XILENQuCN-K8UbAN_XCjemGXjFnzwVSfIooMmEMXzuZWLPL4v/pub?gid=0&single=true&output=csv"
gsheet_hand_corrections_df = pd.read_csv(G_SHEET_URL)

# Convert song_index to integer
gsheet_hand_corrections_df['song_index'] = gsheet_hand_corrections_df['song_index'].astype('Int64')
gsheet_hand_corrections_df

,SONG,Artist,Currently In,proposed,If David thinks its worth changing,song_index
0,Mine,Taylor Swift,Pursuit,Serenade,yes,4399
1,We Found Love,Rihanna,It's Complicated,Serenade,"NOPE. It's subtle, but this song is definitely...",<NA>
2,What's My Name?,Rihanna + Drake,Sexual Confidence,Pursuit,"No. Yes, there's some pursuit. But I really fe...",<NA>
3,Diplomatic Immunity,Drake,Love Song for the Self,Not a Love Song,yes,4775
4,Papi's Home,Drake,Love Song for the Self,Not a Love Song,yes,4999
5,Bang Bang,"Ariana Grande, etc.",Love Song for the Self,Sexual Confidence,yes,4607
6,Questions...?,Taylor Swift,Not a Love Song,It's Complicated,yes,5076
7,Exile,Taylor Swift,It's Complicated,Heartache,yes,4924
8,ME!,Taylor Swift,Serenade,Love Song for the Self,"No. I get that it overlaps a bit, yes. But it'...",<NA>
9,willow,Taylor Swift,Serenade,Heartache,"Nope. This song is complex, but it's definitel...",<NA>


In [10]:
correction_df = gsheet_hand_corrections_df[['song_index', 'proposed']]
correction_df = correction_df[
    correction_df.song_index.notna()
]
CHANGED_TYPES = {
    'Pursuit': 'Courtship & Anticipation',
    'Sexual Confidence': 'Sexual Conquest',
    'Heartache': 'Longing & Heartbreak',
    'Not a Love Song': np.nan,
}
correction_df['proposed'] = correction_df['proposed'].replace(CHANGED_TYPES)
correction_df

,song_index,proposed
0,4399,Serenade
3,4775,NaN
4,4999,NaN
5,4607,Sexual Conquest
6,5076,It's Complicated
7,4924,Longing & Heartbreak
10,5122,Love Song for the Self
11,5079,Love Song for the Self
12,815,NaN
13,1197,NaN


# export

In [11]:
WITH_LOVE_SONG_TYPES_PATH = './data/22.2-v4-OUTPUT-just-the-hand-corrections-to-apply.csv'
correction_df.to_csv(WITH_LOVE_SONG_TYPES_PATH, index=False)